# **Capstone Part II**

In [2]:
import pandas as pd
import numpy as np

In [3]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_train = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

In [39]:
print(aisles.shape)
aisles.head()

(134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [38]:
print(departments.shape)
departments.head()

(21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [37]:
print(order_products_prior.shape)
order_products_prior.head()

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [36]:
print(order_products_train.shape)
order_products_train.head()

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [32]:
print(orders.shape)
orders.head()

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [31]:
print(products.shape)
products.head()

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


![](schema.png)

Blah Blah Blah what is all of this saying??

Kaggle Goal: Predict which previously purchased item will be in a user's next order.

My Goal: Do that^ but also create an algorithm that will recommend items based off of those previously purchased.

How: I can start by trying to predict the high frequency order history customers (customers that have about 50-100 previous orders).  From there I will go lower and lower until I get to predicting a new user.  Then, depending on time, I would like to either spin this off into creating a prediction recomendation algorithm (you bought taco shells... Try this ¡SALSA!).  Ultimately I would love to get started on webscraping and implement that algorithm into that app I was trying to create but for now I will stick with the basics.

**Running some NLP LDA on the aisles to see if i can group these together.  We'll see if it works**

In [9]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [8]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cv = CountVectorizer(stop_words='english')
cv.fit(aisles['aisle'].values)
X = cv.transform(aisles['aisle'].values)
X

<134x201 sparse matrix of type '<class 'numpy.int64'>'
	with 303 stored elements in Compressed Sparse Row format>

In [16]:
feature_names = cv.get_feature_names()
lda = LatentDirichletAllocation(n_topics = 15)

lda.fit(X)

/Users/benjaminseifert/anaconda/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_jobs=1, n_topics=15, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [17]:
print(lda.components_.shape)

(15, 201)


In [18]:
results = pd.DataFrame(lda.components_,
                      columns=feature_names)

In [20]:
for topic in range(15):
    print('Topic', topic)
    word_list = results.T[topic].sort_values(ascending=False).index
    print(' '.join(word_list[0:25]), '\n')

Topic 0
water sparkling desserts broth bakery soup candy bouillon buns vinegars seltzer rolls yogurt chocolate oils cakes cookies meat vegetarian dry shave coolers tapenades fresheners hot 

Topic 1
ice cream fruit soft spirits replacements alternatives breakfast protein applesauce bakery drinks canned meat foods meal tofu asian pizza wines doughs bake herbs frozen mixes 

Topic 2
fresh latino bars foods pastries breakfast vegetables fruits herbs pasta skin sauce care dips dressing kosher popcorn cereal condiments tortillas bowls pizza drink meal baby 

Topic 3
wines specialty champagnes beers digestion coolers lactosefree butter cheeses soy red white toppings cream ice foods fruits cheese packaged fresh instant hot diapers meat formula 

Topic 4
food popcorn soap formula dog jerky care baby frozen breads meat flat milk doughs lunch dry olives plates pastries bread seasonings tapenades sausage energy red 

Topic 5
salads soups granola prepared pancake desserts frozen cat gelatins pain 